**Tableau Calculated Fields Parser**

Extract calculated field names and formulas from Tableau .twb or .twbx files for documentation, auditing, or analysis.

In [ ]:
# Upload file if in Google Colab, or set path manually if running locally
try:
    from google.colab import files
    uploaded = files.upload()
    twb_file = list(uploaded.keys())[0]
except ImportError:
    # Not in Colab, use local file path
    twb_file = "sample/example.twb"  # local file path must be added here

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import re

In [ ]:
def extract_xml(twb_file):
    try:
        ns = {'t': 'http://tableausoftware.com/xml/user'}
        tree = ET.parse(twb_file)
        root = tree.getroot()

        fields = []
        for col in root.findall('.//t:column', ns):
            caption = col.attrib.get('caption') or col.attrib.get('name')
            calc = col.find('t:calculation', ns)
            if calc is not None:
                formula = calc.attrib.get('formula')
                fields.append((caption, formula))

        return fields
    except Exception as e:
        print("XML parse error:", e)
        return []

In [ ]:
fields = extract_xml(twb_file)

In [ ]:
if not fields:
    print("No calculated fields found via XML. Trying Document API...")

    !pip install -q git+https://github.com/tableau/document-api-python.git

    from tableaudocumentapi import Workbook

    wb = Workbook(twb_file)
    for ds in wb.datasources:
        for field in ds.fields.values():
            calc = getattr(field, 'calculation', None)
            if calc:
                fields.append((field.name, calc))
df = pd.DataFrame(fields, columns=["Field Name", "Formula"])
df.head()